<a href="https://colab.research.google.com/github/spegas/Dacon/blob/main/Get_Price_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=0aac2173c28b79e092605c1f3de19f892fa6b78d1fd1a7c2d53fbe7a04fe7ab3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [9]:
import pandas as pd
import os
import FinanceDataReader as fdr
import wget

from zipfile import ZipFile
from os.path import basename

In [7]:
if not os.path.isdir("base"):
  print("base 디렉토리 생성")
  os.makedirs("base")

if not os.path.isdir("price"):
  print("price 디렉토리 생성")
  os.makedirs("price")

In [10]:
stock_list_file_url = "https://raw.githubusercontent.com/spegas/Dacon/main/stock_list.csv"
if os.path.exists('base/stock_list.csv'):
  os.remove('base/stock_list.csv')
  print('이전에 저장된 주식 종목 리스트 파일을 삭제 합니다.')
wget.download(stock_list_file_url, 'base/stock_list.csv')
print('주식 종목 리스트 파일 다운로드 완료')

주식 종목 리스트 파일 다운로드 완료


In [ ]:
stock_list_name = 'base/stock_list.csv'
pd_stock_list = pd.read_csv(stock_list_name)
pd_stock_list['종목코드'] = pd_stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
pd_stock_list.head(10)

In [14]:
start_date = '20200101'
end_date = '20211031'

In [ ]:
for f_index in range(0, pd_stock_list.shape[0]):
  f_code = pd_stock_list.iloc[f_index]['종목코드']
  f_name = pd_stock_list.iloc[f_index]['종목명']
  price_filename = str('price/') + str(f_code) + str('_price.csv')
  if not os.path.exists(price_filename):
    f_price = fdr.DataReader(f_code, start = start_date, end = end_date)
    f_price.to_csv(price_filename)
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 저장 완료' % (str(f_code), str(f_name)))
  else:
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 파일 존재' % (str(f_code), str(f_name)))

In [25]:
# Zip the files from given directory that matches the filter
def zipFilesInDir(dirName, zipFileName, filter):
   # create a ZipFile object
   with ZipFile(zipFileName, 'w') as zipObj:
       # Iterate over all the files in directory
       for folderName, subfolders, filenames in os.walk(dirName):
           for filename in filenames:
               if filter(filename):
                   # create complete filepath of file in directory
                   filePath = os.path.join(folderName, filename)
                   # Add file to zip
                   zipObj.write(filePath, basename(filePath))

In [29]:
zipFilesInDir('price', 'price.zip', lambda name : 'csv' in name)

In [42]:
pd_price = pd.read_csv('price/005380_price.csv')

In [43]:
pd_price

,Date,Open,High,Low,Close,Volume,Change
0,2020-01-02,121000,121000,118000,118000,556532,-0.020747
1,2020-01-03,118500,119000,115000,116000,1145187,-0.016949
2,2020-01-06,116000,117000,115500,116000,334574,0.000000
3,2020-01-07,116500,117000,115500,115500,674196,-0.004310
4,2020-01-08,115000,115500,111500,112000,1044825,-0.030303
...,...,...,...,...,...,...,...
447,2021-10-25,207500,212000,206000,211000,649860,0.016867
448,2021-10-26,213000,214500,209000,213500,782736,0.011848
449,2021-10-27,214500,215000,212500,213500,606729,0.000000
450,2021-10-28,213000,214000,209500,210000,633732,-0.016393


In [44]:
# create 20 days simple moving average column
pd_price['20_SMA'] = pd_price['Close'].rolling(window = 20, min_periods = 1).mean()
pd_price['50_SMA'] = pd_price['Close'].rolling(window = 50, min_periods = 1).mean()

In [45]:
pd_price['20_EMA'] = round(pd_price['Close'].ewm(span = 20, adjust = False).mean(), 2)
pd_price['50_EMA'] = round(pd_price['Close'].ewm(span = 50, adjust = False).mean(), 2)

In [46]:
pd_price

,Date,Open,High,Low,Close,Volume,Change,20_SMA,50_SMA,20_EMA,50_EMA
0,2020-01-02,121000,121000,118000,118000,556532,-0.020747,118000.000000,118000.000000,118000.00,118000.00
1,2020-01-03,118500,119000,115000,116000,1145187,-0.016949,117000.000000,117000.000000,117809.52,117921.57
2,2020-01-06,116000,117000,115500,116000,334574,0.000000,116666.666667,116666.666667,117637.19,117846.21
3,2020-01-07,116500,117000,115500,115500,674196,-0.004310,116375.000000,116375.000000,117433.65,117754.20
4,2020-01-08,115000,115500,111500,112000,1044825,-0.030303,115500.000000,115500.000000,116916.16,117528.55
...,...,...,...,...,...,...,...,...,...,...,...
447,2021-10-25,207500,212000,206000,211000,649860,0.016867,204375.000000,208500.000000,206913.83,210099.97
448,2021-10-26,213000,214500,209000,213500,782736,0.011848,204700.000000,208350.000000,207541.09,210233.30
449,2021-10-27,214500,215000,212500,213500,606729,0.000000,205175.000000,208240.000000,208108.60,210361.41
450,2021-10-28,213000,214000,209500,210000,633732,-0.016393,205600.000000,208070.000000,208288.73,210347.24


In [47]:
pd_price.corr()

,Open,High,Low,Close,Volume,Change,20_SMA,50_SMA,20_EMA,50_EMA
Open,1.000000,0.998445,0.999037,0.997855,-0.086021,-0.027081,0.977844,0.938257,0.984036,0.955863
High,0.998445,1.000000,0.998027,0.999001,-0.047854,0.007958,0.973371,0.931742,0.979992,0.949894
Low,0.999037,0.998027,1.000000,0.998736,-0.101181,-0.012075,0.979479,0.942051,0.985605,0.959204
Close,0.997855,0.999001,0.998736,1.000000,-0.069527,0.022149,0.975894,0.936321,0.982276,0.954000
Volume,-0.086021,-0.047854,-0.101181,-0.069527,1.000000,0.405614,-0.191276,-0.253390,-0.181530,-0.240718
Change,-0.027081,0.007958,-0.012075,0.022149,0.405614,1.000000,-0.064358,-0.064405,-0.058401,-0.062871
20_SMA,0.977844,0.973371,0.979479,0.975894,-0.191276,-0.064358,1.000000,0.976179,0.998804,0.986662
50_SMA,0.938257,0.931742,0.942051,0.936321,-0.253390,-0.064405,0.976179,1.000000,0.977427,0.996823
20_EMA,0.984036,0.979992,0.985605,0.982276,-0.181530,-0.058401,0.998804,0.977427,1.000000,0.987998
50_EMA,0.955863,0.949894,0.959204,0.954000,-0.240718,-0.062871,0.986662,0.996823,0.987998,1.000000
